In [1]:
#Citations :
# https://github.com/yuchenlin/lstm_sentence_classifier/blob/master/LSTM_sentence_classifier.py#L46
# https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html
# https://stackoverflow.com/questions/47952930/how-can-i-use-lstm-in-pytorch-for-classification
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import string
import re

In [35]:
df = pd.read_csv('smallpro.csv',delimiter='\t')

In [38]:
df.tail(50)

,Unnamed: 0,Unnamed: 0.1,To,Subject,content,user,labeled,rep
19950,26225,26225,frozenset({'shona.wilson@enron.com'}),Houston MG Visit,Who are these guys and should someone from our...,beck-s,False,0
19951,34223,34223,"frozenset({'richard.hanagriff@enron.com', 'lin...",New OBA's,The attached OBA's have been submitted to the ...,blair-l,False,0
19952,57616,57616,"frozenset({'james.steffes@enron.com', 'jeff.da...",SCE PX discussion,Wanda Curry had call with SCE Thursday afterno...,dasovich-j,False,0
19953,10420,10420,frozenset({'david.baumbach@enron.com'}),NaN,NaN,bass-e,False,0
19954,87065,87065,frozenset({'pete.davis@enron.com'}),Start Date: 10/19/01; HourAhead hour: 14; <CO...,Start Date: 10/19/01; HourAhead hour: 14; No a...,dean-c,False,0
19955,41194,41194,"frozenset({'timothy.jones@enron.com', 'larry.c...",Fuel Numbers,1999 totals: Unit 301 169393 MCF Unit 302 1433...,campbell-l,False,0
19956,59450,59450,frozenset({'etc.survey@enron.com'}),NO ACTION REQUIRED - TEST,No Action Required - This Is An Email Test - P...,dasovich-j,False,0
19957,31283,31283,frozenset({'leslie.reeves@enron.com'}),Re: Counterparty Names,Thanks for following up on this. From: Leslie ...,beck-s,False,0
19958,74034,74034,"frozenset({'hcameron@uclink.berkeley.edu', 'gr...",Meeting notes,Good luck on the calls everyone. If Monday doe...,dasovich-j,False,0
19959,8206,8206,frozenset({'enronoptions.mailing@enron.com'}),EnronOptions Announcement,EnronOptions Announcement You can now access a...,arora-h,False,0


In [39]:
df = df.drop(['Unnamed: 0','Unnamed: 0.1', 'To','Subject','user','labeled'], axis=1)
# df = df[df.label != 'unsup']

In [42]:
# df['label'] = df['label'].str.lower().replace({'pos': 1, 'neg': 0})
df = df.dropna()
df.tail(60)

,content,rep
19939,"Larry, Here is the DFG-1603 notification cover...",0
19940,custy interest from sell side? or buy? sounds ...,1
19941,Who has the E-schedules for co. 1n2 and 272? M...,0
19942,CALENDAR ENTRY: APPOINTMENT Description: Lynn/...,0
19943,<<NUI-NOM FORMS.xls>> - NUI-NOM FORMS.xls,0
19944,Group- Brace yourselves... Once more we are ex...,0
19945,tom: attached please find the confidential job...,0
19946,yes but getting haircut until 700. is it too d...,0
19947,FYI. Thanks. Lynn -----Original Message----- F...,1
19948,FYI Jim Derrick -----Original Message----- Fro...,1


In [43]:
import argparse
import time, random
from tqdm import tqdm
from torchtext import data
import numpy as np
import argparse
import os

In [44]:
df.content[1]

"Interested in learning more about the fun and convenience of on-line stock trading? Join EFCU for our On-Line Stock Trading Brown Bag. WHO: Joel Spry of Star Financial Network will discuss the basics of on-line trading and EFCU's trading program. WHEN: Friday, September 14, 2001 11:30 - 12:30 WHERE: EB 5C2 RSVP: joy.wagman@enron.com Refreshments will be provided. <Embedded Picture (Metafile)>"

In [45]:
import string
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

exclude = set(string.punctuation)

def cleanup(text):
    text = text.lower()
    text = ''.join(ch for ch in text if ch not in exclude)
    text = ''.join([i for i in text if not i.isdigit()])
    text = text.split()
    texts = []
    for word in text:
        word = lemmatizer.lemmatize(word)
        texts.append(word)
    return texts

[nltk_data] Downloading package wordnet to /Users/ribhu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [46]:
corpus = []
for sent in df.content:
    corpus.append(cleanup(sent))
    
print(corpus[1])

['interested', 'in', 'learning', 'more', 'about', 'the', 'fun', 'and', 'convenience', 'of', 'online', 'stock', 'trading', 'join', 'efcu', 'for', 'our', 'online', 'stock', 'trading', 'brown', 'bag', 'who', 'joel', 'spry', 'of', 'star', 'financial', 'network', 'will', 'discus', 'the', 'basic', 'of', 'online', 'trading', 'and', 'efcus', 'trading', 'program', 'when', 'friday', 'september', 'where', 'eb', 'c', 'rsvp', 'joywagmanenroncom', 'refreshment', 'will', 'be', 'provided', 'embedded', 'picture', 'metafile']


In [47]:
labels = []
for label in df.rep:
    labels.append(label)

In [48]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
wordvec = KeyedVectors.load_word2vec_format(filename, binary=True)

In [49]:
wordvec.get_vector('but')

array([-4.76074219e-02,  8.15429688e-02,  4.56542969e-02,  9.17968750e-02,
       -1.47094727e-02,  1.11328125e-01,  6.54296875e-02, -9.66796875e-02,
        1.38671875e-01,  1.43554688e-01, -7.86132812e-02, -1.63085938e-01,
       -9.17968750e-02, -7.12890625e-02, -8.05664062e-02,  8.83789062e-02,
        2.17773438e-01,  9.96093750e-02,  1.19018555e-02, -1.35742188e-01,
       -5.78613281e-02,  9.03320312e-03,  1.75781250e-01, -1.17187500e-02,
        4.90722656e-02, -1.05468750e-01, -1.53320312e-01,  6.17675781e-02,
        8.64257812e-02,  2.25830078e-02,  1.59740448e-05,  6.39648438e-02,
        2.33154297e-02, -1.45874023e-02,  1.51367188e-02,  5.29785156e-02,
       -1.34277344e-02,  7.99560547e-03,  2.63671875e-02,  2.99072266e-02,
        1.61132812e-01,  5.68847656e-02,  1.18164062e-01, -3.12500000e-02,
       -7.59887695e-03,  6.64062500e-02, -1.08032227e-02, -6.53076172e-03,
        1.13281250e-01,  9.22851562e-02,  4.29687500e-02, -6.10351562e-03,
        2.80761719e-02, -

In [50]:
#Splitting corpus of sentences into train, test, validation
sen_vec = []
X_train = []
X_cv = []
X_test = []
corp_train = corpus[0:200]
corp_dev = corpus[200:300]
corp_test = corpus[300:400]
for sentence in corp_train:
    for word in sentence:
        try:
            sen_vec.append(wordvec.vocab[word].index)
        except KeyError:
            sen_vec.append(0) #If word not in vocab add tensor of 1s so that it doesnt affect the result
    input_vec = torch.LongTensor(sen_vec)
    X_train.append(input_vec)
    sen_vec = []
for sentence in corp_dev:
    for word in sentence:
        try:
            sen_vec.append(wordvec.vocab[word].index)
        except KeyError:
            sen_vec.append(0) #If word not in vocab add tensor of 1s so that it doesnt affect the result
    input_vec = torch.LongTensor(sen_vec)
    X_cv.append(input_vec)
    sen_vec = []
for sentence in corp_test:
    for word in sentence:
        try:
            sen_vec.append(wordvec.vocab[word].index)
        except KeyError:
            sen_vec.append(0) #If word not in vocab add tensor of 1s so that it doesnt affect the result
    input_vec = torch.LongTensor(sen_vec)
    X_test.append(input_vec)
    sen_vec = []
                                    #Should try with randoms as a variation if shit don't work



In [51]:
#Splitting labels in the same
labels_train = labels[0:200]
labels_cv = labels[200:300]
labels_test = labels[300:400]

In [52]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(wordvec.vectors))
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        y = self.hidden2tag(lstm_out[-1])
        log_probs = F.log_softmax(y,dim=1)
        return log_probs

In [53]:
def train():
    EMBEDDING_DIM = 300
    HIDDEN_DIM = 150
    EPOCH = 20
    best_dev_acc = 0.0
    train_size = len(labels_train)
    dev_size = len(labels_cv)
    test_size= len(labels_test)
    model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, 300000, 2)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    loss_function = nn.NLLLoss()
    optimizer = optim.Adam(parameters,lr = 1e-3)
    #optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2)
    no_up = 0
    for i in range(EPOCH):
        print('epoch: %d start!' % i)
        train_epoch(model, train_size, loss_function, optimizer, X_train, labels_train, i)
        print('now best dev acc:',best_dev_acc)
        dev_acc = evaluate(model,dev_size,loss_function,X_cv, labels_cv,'dev')
        test_acc = evaluate(model, test_size, loss_function, X_test, labels_test, 'test')
        if dev_acc > best_dev_acc:
            best_dev_acc = dev_acc
            os.system('rm mr_best_model_acc_*.model')
            print('New Best Dev!!!')
#             torch.save(model.state_dict(), 'best_models/mr_best_model_acc_' + str(int(test_acc*10000)) + '.model')
            no_up = 0
        else:
            no_up += 1
            if no_up >= 10:
                exit()

In [54]:
def get_accuracy(truth, pred):
    assert len(truth)==len(pred)
    right = 0
    for i in range(len(truth)):
        if truth[i]==pred[i]:
            right += 1.0
    return right/len(truth)

def train_epoch(model, datapoints, loss_function, optimizer, text, labels, i):
    model.train()
    
    avg_loss = 0.0
    count = 0
    truth_res = []
    pred_res = []
    batch_sent = []
    
    truth_res = labels

    for j in range(datapoints):
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()
        sent = text[j]
        label = torch.tensor(labels[j], dtype=torch.long)
        pred = model(sent)
        pred_label = pred.data.max(1)[1].numpy()
        pred_res.append(pred_label)
        model.zero_grad()
        loss = loss_function(pred, label.unsqueeze_(0))
#         print(type(loss.data.item()))
        avg_loss += loss.data.item()
        count += 1
        if count % 500 == 0:
            print('epoch: %d iterations: %d loss :%g' % (i, count, loss.data[0]))

        loss.backward()
        optimizer.step()
    avg_loss /= datapoints
    print('epoch: %d done! \n train avg_loss:%g , acc:%g'%(i, avg_loss, get_accuracy(truth_res,pred_res)))

def evaluate(model, datapoints, loss_function, text, labels, name ='dev'):
    model.eval()
    avg_loss = 0.0
    truth_res = labels
    pred_res = []

    for j in range(datapoints):
        # detaching it from its history on the last instance.
        model.hidden = model.init_hidden()
        sent = text[j]
        label = torch.tensor(labels[j], dtype=torch.long)
        pred = model(sent)
        pred_label = pred.data.max(1)[1].numpy()
        pred_res.append(pred_label)
        # model.zero_grad() # should I keep this when I am evaluating the model?
        loss = loss_function(pred, label.unsqueeze_(0))
        avg_loss += loss.data.item()
    avg_loss /= datapoints
    acc = get_accuracy(truth_res, pred_res)
    print(name + ' avg_loss:%g train acc:%g' % (avg_loss, acc ))
    return acc

In [55]:
train()

epoch: 0 start!
epoch: 0 done! 
 train avg_loss:0.472254 , acc:0.85
now best dev acc: 0.0
dev avg_loss:0.587671 train acc:0.75
test avg_loss:0.527371 train acc:0.79
New Best Dev!!!
epoch: 1 start!
epoch: 1 done! 
 train avg_loss:0.399688 , acc:0.85
now best dev acc: 0.75
dev avg_loss:0.61239 train acc:0.75


KeyboardInterrupt: 